<a href="https://www.kaggle.com/code/peremartramanonellas/notebook-with-explanations-deal-with-the-data?scriptVersionId=105293144" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

As you know this is based in the competition **House Prices - Advanced Regression Techniques** 

What can be found in the notebook? 
- Identify columns with few values that we can drop. 
- How to treat the NULL values. 
- Study the data with BoxPlots, scatterplots, heatmaps... 
- Identify the most correlated features.
- Identify and correct the skew values. 
- Transform all the data to numeric. 
- Categorize the data. 
- Scale the numeric data, if necesary, with StandarScaler or MinMaxScaler. 
- Use a model, get predictions and submit results. 

I did my best to explain each step. If you have any comment or question, please don't hesitate to use the comments section. 

- <a href='#sectiondata'>Data</a>

  - <a href='#cleannulls'>Clean Nulls</a>

    - <a href='#heatmap'>HeatMap Garage</a>

  - <a href='#scaling'>Scaling</a>

  - <a href='#correlated'>Select correlated features</a>

- <a href='#training'>Create & Training the model</a>

  - <a href='#submission'>Submission</a>
 
- <a href='#inspirations'>Inspirations</a>



In [ ]:
#Import libraries 

import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

%matplotlib inline   

In [ ]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

#I'm going to join the train & Test Data in order to do the same transformations. 
df_housing=pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

#keeping the number of elements in the train dataframe. 
TRAIN_ELEMENTS_INDEX = df_train.index.max()

df_train.shape, df_test.shape, df_housing.shape


# Data.<a id='sectiondata'></a> 
Actions that we should do with the Data: 
- Identify columns with unique values.  
- Clean / Remove / Replace the Nulls. 
- Replace all the non numeric values
- Check for outliers
- Standarize / Normalize or adapt the numeric values.

 

In [ ]:
#Just a quick look to our data. 
df_housing.head(20)

As you can see, we have some columns with just a few values, let's check their *uniquevalues* and take a decisión if we want to keep, or not, the column. 

In [ ]:
#check the number of elements in each unique value in the columns. c
COLS_TO_CHECK = ['Street', 'Utilities', 'Condition2', 
                'LandSlope', 'RoofMatl', 'ExterCond', 'Heating', 
                'CentralAir', 'GarageCond', 'Electrical']

#I create a DataFrame where store the Data
dftemp = pd.DataFrame(columns=['col_name', 'value', 'count'])

#In every colum to check I get the values and number of them, 
#and store in the DataFrame
for ctc in COLS_TO_CHECK: 
    dcol = df_housing[ctc].value_counts()
    for i in range(len(dcol)):
        row = {'col_name': ctc, 
              'value': dcol.index[i], 
              'count': dcol[i]}
        df_row = pd.DataFrame([row])
        dftemp = pd.concat([dftemp, df_row], axis=0, ignore_index=True)

In [ ]:
#A list of all the values, it's easy to identify some that we can drop. 
dftemp

It's clear to me that we can Drop *Street* and *Utilities* without any doubt. In *Utilities* we have just two values and one of them is present in only one row. 

In [ ]:
df_housing.drop(['Street', 'Utilities'], axis=1, inplace=True)

## Clean the Nulls.<a id='cleannulls'></a> 

In [ ]:
#change the ppandas visualization options to view all columns an rows if necessary. 
pd.set_option('display.max_rows', 1000); pd.set_option('display.max_columns', 1000); 
df_housing.shape

We have 2919 rows by 79 columns in the train Dataset . 

In [ ]:
df_housing.head(5)

In [ ]:
#Get the numbers of Null values in columns, in descending order. 
df_housing.isna().sum().sort_values(ascending=False).head(40
                                                         )

There are columns with a lot of empty rows. Usually **the best that we can do with the columns with a big number of empty data is delete it**, because we can't fill with invented values. 

To deal with empty data we have some different options how to deal with empty values: 

**Delete the column:** When the number of rows with empty values is really big, the best option is delete the entire column. I have no a magic number, but if it's more than 80% of rows with empty values I normally delete the column. 

**Replace the values:** Sometimes there are empty values because it indicates a category, and this category dosn't apply to some of the registers. 
That's the case of Fence in this dataset. There are a lot of properties without a Fence, but the properties with a Fence have different categories of Fences that affect to the price of this properties. In this case we can fill the empty row with the category *No. 

**Delete the rows**: When the empty is in a feature with a strong correlation with the label, the best solution is delete the rows. But in this case I'm going to avoid it, because I Joined the train and test dataset. To be able to delete rowns I should have joined them with a different technique. 

**By the moment, we are going to delete the columns:PoolQC. MiscFeature,  Alley and Id.**


In [ ]:
df_house_clean = df_housing.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley'], axis=1)

In [ ]:
plt.title('Fence ')
#Note that I'm getting only the Train part of the Dataframe, the test have no SalePrice
sns.boxplot(x='Fence', y='SalePrice', data=df_house_clean[:TRAIN_ELEMENTS_INDEX:])

I can't se a clear relathionship between Fence and the SalePrice, and there are a lot of outliers values. We can just delete this value too. 

In [ ]:
df_house_clean.drop(['Fence'], axis=1, inplace=True)

In [ ]:
#Check the unique values in FirePlaceQu
df_house_clean['FireplaceQu'].unique()

I'm going to fill the Null Values with a "No", just to have more information in the subplot

In [ ]:
df_house_clean['FireplaceQu'] = df_house_clean['FireplaceQu'].fillna('No')
plt.title('FireplaceQu ')
sns.boxplot(x='FireplaceQu', y='SalePrice', data=df_house_clean[:TRAIN_ELEMENTS_INDEX:])

Ok, it seems that it can be a correlation between the type of fireplace and the *SalePrice*, but it's not strong enough and there are a lot of outliners. I'm not going to keep it. 

Time to check the values in *LotFrontage*. It contains the linear feet of street connected to property. As a resident in a City in Europe I have no Idea if this data can be important or not in the local Market of the Dataset. By the moment we can fill the values with the mean. 

In [ ]:
df_house_clean['LotFrontage'].unique()

In [ ]:
plt.title('LotFrontage ')
sns.scatterplot(x='LotFrontage', y='SalePrice', data=df_house_clean[:TRAIN_ELEMENTS_INDEX:])

I cant see a clear relation between the *LotFrontage* and the *SalePrice*, I'm  going to delete this value too. 


In [ ]:
df_house_clean.drop(['LotFrontage'], axis=1, inplace=True)

Let's check again the columns with empty values
df_house_clean.isna().sum().sort_values(ascending=False).head(15)

In [ ]:
df_house_clean.isna().sum().sort_values(ascending=False).head(15)

We still have a lot of columns with just a few Null values. But we need to decide what to do with them. It's important to know if the data is just numeric or it can be categorical. And when we know it, fill it, or maybe just remove the rows, or columns. 

We have 5 Garage variables with 81 registers with Nulls, but we have others columns related to the garage without missing data. To decide if we can Delete the columns, the rows or fill it. We can study the correlation of all the garage columns, and try to identify wich one is more important to our model. 

At this moment some of this columns are not numeric, and we can calculate the Correlation. We can wait until all our Data is numeric, but I prefer to identify now if we can delete some of this data. 


### Heatmap Garage variables<a id='heatmap'></a>

In [ ]:
#saleprice correlation matrix with Garage Variables. 
cols = ['SalePrice','GarageType', 'GarageYrBlt', 'GarageCond', 
       'GarageFinish', 'GarageQual', 'GarageCars', 'GarageArea']

corrmat = df_house_clean[:TRAIN_ELEMENTS_INDEX:][cols].corr()

f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, annot=True, cmap="Blues");

By the moment we can delete *GarageYrBlt*, and keep *GarageArea* and *GarageCars*, but for the name for sure that this Data is redundant. And if I have to choose one, I will choose *GarageArea*, but in the HeatMap we can see that *GarageCars* is more correlated, so....I'm going to Keep *GarageCars* (I believe in numbers). Maybe some garages have a large Area but it's not possible to fit more than one car. 

In [ ]:
#Delete the columns
df_house_clean.drop(['GarageYrBlt', 'GarageArea'], axis=1, inplace=True)


#Fill the column
df_house_clean = df_house_clean.fillna(df_house_clean.mode().iloc[0])


Let's check if we have more Nulls: 

In [ ]:
df_house_clean.isna().sum().sort_values(ascending=False).head(15)

Perfect, **we have no Nulls**. 

## Check the skew of the numeric values. 

In [ ]:

#obtain all the numeric columns 
numeric_columns = list(df_house_clean.select_dtypes(['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns)
numeric_columns




In [ ]:
skew_features = df_house_clean.select_dtypes(['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).apply(lambda x: skew(x)).sort_values(ascending=False)


In [ ]:
high_skew = skew_features[skew_features > 0.5]

#in skew_index we store the name of the columns
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))

#Store the skews colums in a DataFrame
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

In [ ]:
for col in skew_index: 
    if (col != 'SalePrice'):
        df_house_clean[col] = boxcox1p(df_house_clean[col], 
                                   boxcox_normmax(df_house_clean[col]+1))
    

## Transform Data to numeric. 
All the data must be numeric. And we have some columns that are in text. 

We have two ways to convert datas in numeric. 
- In the same column. We can assign a number to each category. But it have a problem. If we assign the numbers 1, 2 and 3 to three diferent categories. Our model will be confused because 2 is bigger than 1 and 3 bigger than 2, but it isn't important, we are just categorizing. I only recommend this kind of categorization in fields where bigger values have more impact in the label than the smaller ones. If we have a Column called GarageSize with the values small, medium, and big. We can convert this categories into 1, 2 and 3 (or a normalized value), because the size of the garage have a positive correlation with the prices of property. 

- In Categories. We create a new column for each value, and indicate if the row have this value with an 1 or a 0. This the most commom way to transform from text to numeric, but the text must be categorizable. Some times we need to create our own categories if we want to keep the data. As a sample, colors of a car, can be a lot of them, impossible to categorize, but we can distribute it in: Dark colors and clear colors, or maybe metallic colors. But it needs a lot of manual revisión.  


First we need to know with colums are not numeric. 

In [ ]:
df_house_clean.loc[:, df_house_clean.dtypes == object]

In [ ]:
object_columns = list(df_house_clean.select_dtypes(['object']).columns)
object_columns

We have 40 columns to transform, if we want to categorize all of them and we have 5 categories per column we will add 160 columns to the model.  

(40 * 5 = 200) - 40 = 160. 

It seems a lot, but it's not a problem. The problem is that not all of them are relevant, and we are fitting our model with a lot of useless informatión. But let's try it, we can delete it after all columns are converted when we can study the correlation of all columns. 

Just one sample with the column *LotShape*

In [ ]:
df_housing['LotShape'].unique()

In [ ]:
ls = pd.get_dummies(df_house_clean['LotShape'], prefix='LotShape')
ls

As you can see, we have a new column for each categorie, and  every registry have a value of *1* in the category where it pertains. 

The next step is add this columns to our dataframe, and delete the original one, because we don't need it anymore. 

Now, we can do it for all the columns with data values in our Dataset, as we can see above in the list all of them are categorizables. 

In [ ]:
df_house_clean.shape

In [ ]:
#Copy the dataframe. 
df_house_cc = df_house_clean.copy(deep = True)

#Create the dummies for each column in the list of Obcjet columns
for obj_col in object_columns: 
    #print(obj_col)
    col_dum = pd.get_dummies(df_house_clean[obj_col], prefix=obj_col)
    df_house_cc = pd.concat ([df_house_cc,  col_dum], axis=1)
    
    #remove the original columns from the new dataframe
    df_house_cc.drop(obj_col, axis=1, inplace=True)

In [ ]:
df_house_cc.shape

In [ ]:
df_house_cc.isna().sum().sort_values(ascending=False).head(5)

In [ ]:
#Create a duplicate of the dataframe witout the Data Scaled. 
df_not_scaled = df_house_cc.copy(deep = True)

df_not_scaled.head()

## Scaling the numeric data <a id='scaling'></a>
We have our dataframe without nulls. But we need to do some more transformations in order to improve the data, and do it more usable. 

Now we are going to scalete, if necessary, the Data. 

We can do it in many ways, but we are using SciKitLearn, and we have two classes to do it: 

- **StandardScaler**: Will transform the numerical fields so that the mean of the field is 0 and the standard deviation is 1. 
- **MinMaxScaler**: scale the columns so that the minimum value of each column is 0 and the maximum value is 1. 

With this DataSet I think that StandardScaler is going to work better than MinMaxScaler, but feel free to check it by yourself. 

In [ ]:
#We can add this line to the import Libraries section. 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

scaler = StandardScaler()

The easy way to scale: 

* scaler = StandardScaler() *

* df_to_scale_tmp = scaler.fit_transform(df_house_clean) *

* df_house_clean = pd.DataFrame(df_to_scale_tmp, columns=df_to_scale.columns) *

But in this way we scalate all the values un the dataset. I'm doing the same, but selecting first with columns I want to scale and copy it to a temporaru Dataset. We can select the columns manually, but I decided to Sacalate all the columsn with a std greater than 1. But this value is in a constant and I can do a lot of test changing this limit. 

The SalePrice deserves a special treatment. Is our label variable, and we must assure that it is following a normal distribution. 

In [ ]:
sns.displot(df_house_cc[:TRAIN_ELEMENTS_INDEX:]['SalePrice']);
fig = plt.figure()
res = stats.probplot(df_house_cc[:TRAIN_ELEMENTS_INDEX:]['SalePrice'], plot=plt)

In [ ]:
#applying log transformation
df_house_cc['SalePrice'] = np.log(df_house_cc['SalePrice'])

In [ ]:
sns.displot(df_house_cc[:TRAIN_ELEMENTS_INDEX:]['SalePrice']);
fig = plt.figure()
res = stats.probplot(df_house_cc[:TRAIN_ELEMENTS_INDEX:]['SalePrice'], plot=plt)

Now that our Label is Normalized we can attack all the other columns. Sometimes is better to scale all of them, sometimes only some of them. Thas why I prepared this code where I can select the level of std desviation that I want to use as a limit to select the variables to Scale. 

And I can do experiments changing this value and check how it affetct to the final predictions. 

In [ ]:
#We need to decide wich columns we want to scalate
#all of them with and std greater than 1 
LIMIT_TO_SCALE = 1
columns_to_scale = df_house_cc.columns.where(df_house_cc.std()  > 1)

df_to_scale = pd.DataFrame()

#We have all the columns in columns_to_scale
#we can copy each column in a new dataframe. 
for col_to_scale in columns_to_scale.dropna():
    df_to_scale[col_to_scale] = df_house_cc[col_to_scale]
    df_house_cc.drop(col_to_scale, axis=1, inplace=True)

In [ ]:
df_house_cc.shape, df_to_scale.shape

We have two dataframes: 
- df_to_scale: contains all the columns that we want to scale. 
- df_house_cc: is the original dataframe with the data Clean and Converted to numeric. We don't want to normalize all this dataframe because the categorical columns. 

Now it's time to normalize df_to_scale and replace the values in df_house_cc.

In [ ]:
#Let's see the data in df_to_scale befote to scale. 
df_to_scale.head(5)

Scalate the Data is really easy, now that we have the all the Data that we need to Scalate in a Dataframe, we only need 3 lines of code: 

In [ ]:
#Create the Scaler
scaler = StandardScaler()

#Fit the Scaler with the data. 
df_to_scale_tmp = scaler.fit_transform(df_to_scale)

#Convert to dataframe the value returned by fit_transfom function. 
df_to_scale = pd.DataFrame(df_to_scale_tmp, columns=df_to_scale.columns)

In [ ]:
#After scale. 
df_to_scale.head()

In [ ]:
#Replace the values not scaled by the scaled versión. 
for col_to_scale in columns_to_scale.dropna():
    df_house_cc[col_to_scale] = df_to_scale[col_to_scale]

In [ ]:
df_house_cc.shape

In [ ]:
#Lets see the data scaled jointly with all the other fields. 
df_house_cc.head(5)

## Select correlated features in a new dataset <a id='correlated'></a>
Nom that we have all the columns in numeric we can select the ones more correlated. But we are going to keep the dataset transformed but with all the columns to test it with the models. 

In [ ]:
list_columns_ordered = df_house_cc.corr()['SalePrice'].sort_values(ascending=False).index
n=0

#I create a new DataFrame to contain only the columns that are correlated. 
df_correlated_columns = pd.DataFrame()

#Minimun correlation that we want. 
MIN_CORR = 0.4
for col in df_house_cc[:TRAIN_ELEMENTS_INDEX:].corr()['SalePrice'].sort_values(ascending=False):
    if (col > MIN_CORR or col < MIN_CORR * -1):
        print (list_columns_ordered[n])
        print (col)
        df_correlated_columns[list_columns_ordered[n]] = df_house_cc[list_columns_ordered[n]]
    
    n = n+1


In [ ]:
df_correlated_columns.head()

Ok! **OUR DATA IS READY!** Let's use it! 

I have 3 different dataframes to test: 
- **df_correlated_columns**: A dataframe with all the modifications, and only with the columns witha correlation > 5. 

- **df_house_cc**: Without nulls and with all data in numeric and standarized, but with all the columns, not only the more correlated. 

- **df_not_scaled**: Without nuls and with all data in numeric but not standarized. 

And their respective test datasets. 


Just, one last step: Split the data. 

In [ ]:
from sklearn.model_selection import train_test_split

#Split the data only correlated columns. 
y = df_correlated_columns[:TRAIN_ELEMENTS_INDEX:]['SalePrice']
X = df_correlated_columns[:TRAIN_ELEMENTS_INDEX:].drop(['SalePrice'], axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = df_correlated_columns.loc[TRAIN_ELEMENTS_INDEX + 1:, :].reset_index(drop=True).copy()
X_test.drop('SalePrice', axis=1, inplace=True)

#Split data with all transformations and all columns. 
y_cc = df_house_cc[:TRAIN_ELEMENTS_INDEX:]['SalePrice']
X_cc = df_house_cc[:TRAIN_ELEMENTS_INDEX:].drop(['SalePrice'], axis=1)
X_cc_train, X_cc_val, y_cc_train, y_cc_val = train_test_split(X_cc, y_cc, test_size=0.2, random_state=42)
X_cc_test = df_house_cc.loc[TRAIN_ELEMENTS_INDEX + 1:, :].reset_index(drop=True).copy()
X_cc_test.drop('SalePrice', axis=1, inplace=True)


In [ ]:
X_cc_test.shape

# The Model & Test different DataFrames. <a id='training'></a>
I'm going to keep this part simple. It's a lot that you can do if you want to copy this notebook and improve the results obtained. You can test new models, I only test the ridge. Tune the hyperparameters, or maybe try blended models. 

In [ ]:
def evaluate_regression(y_true, y_preds):
    from sklearn.metrics import r2_score
    """
    Evaluar modelo de regresion
    """
    r2_score = r2_score(y_true, y_preds)
    rmse=np.sqrt(mean_squared_error(y_true,y_preds))
    mae=mean_absolute_error(y_true, y_preds)

    print(f"KPIs-------------------------------------")
    print(f"r2: {r2_score * 100:.2f}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"KPIs-------------------------------------")
    return 

In [ ]:
from sklearn import linear_model
#from sklearn import 
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler



In [ ]:
def train_eval_ridge(X_train, y_train, X_val, y_val):  
    model = linear_model.Ridge()
    model.fit(X_train, y_train)
    print(model.score(X_val, y_val))
    y_preds = model.predict(X_val)
    evaluate_regression(y_val, y_preds)
    return model 

In [ ]:
model1 = train_eval_ridge(X_train, y_train, X_val, y_val)

In [ ]:
model2 = train_eval_ridge(X_cc_train, y_cc_train, X_cc_val, y_cc_val)

In [ ]:
#test_features = df_test_cc[:]
#print (test_features)
results1 = model1.predict(X_test)
results2 = model2.predict(X_cc_test)

## Submission <a id='submission'></a>

In [ ]:
submission = pd.DataFrame()

submission['Id'] = df_test['Id']
submission['SalePrice'] = np.exp(results2)


submission.to_csv('./submission.csv', index=False, header=True)

In [ ]:
submission

# Inspirations: <a id='inspirations'></a>

https://www.kaggle.com/code/serigne/stacked-regressions-top-4-on-leaderboard
https://www.kaggle.com/code/apapiu/regularized-linear-models/notebook
https://www.kaggle.com/code/modassirafzal/housing-top-3
https://www.kaggle.com/code/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition
https://www.kaggle.com/code/pmarcelino/comprehensive-data-exploration-with-python/notebook


# # Feel free to copy and fork this notebook. 
I'm sure that you can improve easily the result of this notebook just working with the Model, or maybe changing the number of colums to scalate, the limit for the skew, or the limit to obtain the correlated columns. Or maybe using Mutual Information instead of correlation. 

Create new features, and delete some of the used... you can test a lot of ideas for your own! 
***
Just, if you like it, **please consider to upvote**! 
Please, if you improve the results of the predictions using any technique described, consider to share your notebook in the comments i will be delighted to explore and discuss your solution, and of course upvote it! 
***

Thanks! 

May the Data be with you ;-) 